In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_feather("dataFrame.feather")
data["category"] = pd.Categorical(data["category"])
print(data.shape)
X = data["text"]
Y = pd.get_dummies(data["category"])
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.2)

(2225, 3)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# get pairs word:index(word) 
vocabulary = {token: token_index for token_index, token in enumerate(model_w2v.wv.index2word)} 
# change words to integer seq using indexes
tk = Tokenizer(num_words=len(vocabulary)) 
tk.word_index = vocabulary 

train_text_seq = tk.texts_to_sequences(train_X)
maxlen = max([len(text) for text in train_text_seq])
train_text_seq = pad_sequences(train_text_seq, maxlen=maxlen, padding="post", value=0.0)

test_text_seq = tk.texts_to_sequences(test_X)
test_text_seq = pad_sequences(test_text_seq, maxlen=maxlen, padding="post", value=0.0)

